In [1]:
from time import time
from typing import Callable, Dict, Set

import numpy as np
import onnx
import tensorrt as trt
import torch
from onnx import ModelProto
from tensorrt import ICudaEngine
from tensorrt.tensorrt import Logger, Runtime
from torch.nn import Linear
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, PretrainedConfig, T5ForConditionalGeneration, TensorType
from transformers.generation_utils import GenerationMixin
from transformers.modeling_outputs import BaseModelOutputWithPastAndCrossAttentions, Seq2SeqLMOutput
from transformers.models.t5.modeling_t5 import T5Stack

from transformer_deploy.backends.ort_utils import create_model_for_provider, inference_onnx_binding, optimize_onnx
from transformer_deploy.backends.pytorch_utils import convert_to_onnx
from transformer_deploy.backends.trt_utils import (
    TensorRTShape,
    add_output_nodes,
    build_engine,
    get_adjency_dict,
    get_fix_fp16_network_func,
    get_list_fp32_nodes,
    load_engine,
    save_engine,
)

In [2]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
input_ids: torch.Tensor = tokenizer("Studies show that", return_tensors=TensorType.PYTORCH).input_ids
input_ids = input_ids.to("cuda")
model: T5ForConditionalGeneration = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.eval()
model = model.to("cuda")
out_enc: BaseModelOutputWithPastAndCrossAttentions = model.encoder(input_ids=input_ids)
out_full: Seq2SeqLMOutput = model(input_ids=input_ids, decoder_input_ids=input_ids)

In [3]:
class ExportT5(torch.nn.Module):
    def __init__(self, decoder: T5Stack, lm_head: Linear):
        super(ExportT5, self).__init__()
        self.decoder = decoder
        self.lm_head = lm_head

    def forward(self, input_ids: torch.Tensor, encoder_hidden_states: torch.Tensor):
        out_dec = self.decoder.forward(input_ids=input_ids, encoder_hidden_states=encoder_hidden_states)
        # Rescale output before projecting on vocab
        out_dec = out_dec["last_hidden_state"] * (model.model_dim**-0.5)
        out_lm = self.lm_head(out_dec)
        return out_lm

model = model.to("cuda")

convert_to_onnx(
    model_pytorch=model.encoder,
    output_path="test-enc.onnx",
    inputs_pytorch={"input_ids": input_ids},
    var_output_seq=True,
    quantization=False,
)
optimize_onnx(
    onnx_path="test-enc.onnx", onnx_optim_model_path="test-enc-opt.onnx", architecture="bert", use_cuda=True, fp16=True
)

enc_onnx = create_model_for_provider("test-enc-opt.onnx", "CUDAExecutionProvider")
enc_onnx_out = inference_onnx_binding(
    model_onnx=enc_onnx,
    inputs={"input_ids": input_ids},
    device=input_ids.device.type,
    output_shape=tuple(input_ids.shape) + (int(model.encoder.config.d_model),),
)["output"]
assert np.allclose(enc_onnx_out.detach().cpu().numpy(), out_enc.last_hidden_state.detach().cpu().numpy(), atol=1e-2)

model_to_export = ExportT5(decoder=model.decoder, lm_head=model.lm_head).eval()
out_model_export: torch.Tensor = model_to_export(input_ids=input_ids, encoder_hidden_states=out_enc.last_hidden_state)
assert np.allclose(out_model_export.detach().cpu().numpy(), out_full.logits.detach().cpu().numpy(), atol=1e-5)

inputs_onnx = {"input_ids": input_ids, "encoder_hidden_states": out_enc.last_hidden_state}

convert_to_onnx(
    model_pytorch=model_to_export,
    output_path="test-dec.onnx",
    inputs_pytorch=inputs_onnx,
    var_output_seq=False,
    quantization=False,
    fix_output_dim_size=False,  # specific to decoder part
)
optimize_onnx(
    onnx_path="test-dec.onnx",
    onnx_optim_model_path="test-dec-opt.onnx",
    architecture="bert",
    use_cuda=True,
    fp16=True,
    num_attention_heads=model.config.num_heads,
    hidden_size=model.config.d_model,
)

/home/geantvert/.local/share/virtualenvs/fast_transformer/lib/python3.9/site-packages/transformers/modeling_utils.py:529: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if causal_mask.shape[1] < attention_mask.shape[1]:


In [4]:
model = model.to("cuda")
enc_onnx = create_model_for_provider("test-enc-opt.onnx", "CUDAExecutionProvider")
dec_onnx = create_model_for_provider("test-dec-opt.onnx", "CUDAExecutionProvider")


def decoder_pytorch_inference(input_ids: torch.Tensor, last_hidden_state: torch.Tensor):
    out_dec = model.decoder(input_ids=input_ids, encoder_hidden_states=last_hidden_state)["last_hidden_state"]
    # Rescale output before projecting on vocab
    out_dec = out_dec * (model.model_dim**-0.5)
    out_lm = model.lm_head(out_dec)
    return out_lm


def decoder_onnx_inference(input_ids: torch.Tensor, last_hidden_state: torch.Tensor):
    result_dict = inference_onnx_binding(
        model_onnx=dec_onnx,
        inputs={"input_ids": input_ids, "encoder_hidden_states": last_hidden_state},
        device=input_ids.device.type,
        output_shape=tuple(input_ids.shape) + (int(model.config.vocab_size),),
    )
    return result_dict["output"]


def decoder_onnx_standard_inference(input_ids: torch.Tensor, last_hidden_state: torch.Tensor):
    result_list = dec_onnx.run(
        None, {"input_ids": input_ids.type(torch.int32).numpy(), "encoder_hidden_states": last_hidden_state.numpy()}
    )
    return torch.from_numpy(result_list[0])


dec_onnx_out = decoder_onnx_inference(input_ids=input_ids, last_hidden_state=out_enc.last_hidden_state)
assert np.allclose(dec_onnx_out.detach().cpu().numpy(), out_full.logits.detach().cpu().numpy(), atol=1e-1)


def encoder_onnx_inference(input_ids: torch.Tensor, **_) -> BaseModelOutputWithPastAndCrossAttentions:
    result = inference_onnx_binding(
        model_onnx=enc_onnx,  # noqa: F821
        inputs={"input_ids": input_ids},
        output_shape=tuple(input_ids.shape) + (int(model.encoder.config.d_model),),
        device=input_ids.device.type,
    )
    return BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=result["output"])


def encoder_pytorch_inference(input_ids, **_) -> BaseModelOutputWithPastAndCrossAttentions:
    return model.encoder(input_ids=input_ids)


# https://github.com/NVIDIA/TensorRT/blob/main/demo/HuggingFace/T5/export.py
class ExtT5(torch.nn.Module, GenerationMixin):
    def __init__(self, config: PretrainedConfig, device: torch.device, encoder_func: Callable, decoder_func: Callable):
        super(ExtT5, self).__init__()
        self.main_input_name = "input_ids"  # https://github.com/huggingface/transformers/pull/14803
        self.config: PretrainedConfig = config
        self.device: torch.device = device

        self.encoder_func = encoder_func
        self.decoder_func = decoder_func

    def get_encoder(self):
        return self.encoder_func

    def get_decoder(self):
        return self.decoder_func

    def prepare_inputs_for_generation(self, input_ids, **kwargs):
        return {
            self.main_input_name: input_ids,
            "encoder_hidden_states": kwargs["encoder_outputs"]["last_hidden_state"],
        }

    def forward(self, input_ids: torch.Tensor, encoder_hidden_states: torch.Tensor, **_):
        dec_output = self.get_decoder()(input_ids=input_ids, last_hidden_state=encoder_hidden_states)
        return Seq2SeqLMOutput(logits=dec_output)


model_gen = (
    ExtT5(
        config=model.config,
        device=model.device,
        encoder_func=encoder_onnx_inference,  # encoder_pytorch_inference
        decoder_func=decoder_onnx_inference,  # decoder_pytorch_inference
    )
    .cuda()
    .eval()
)

# model = model.eval()
with torch.inference_mode():
    out_enc: BaseModelOutputWithPastAndCrossAttentions = model.encoder(input_ids=input_ids)
    a = model_gen(input_ids=input_ids, encoder_hidden_states=out_enc.last_hidden_state).logits
    b = model(input_ids=input_ids, decoder_input_ids=input_ids).logits
    assert np.allclose(a.detach().cpu().numpy(), b.detach().cpu().numpy(), atol=1e-1)

    print(
        tokenizer.decode(
            model_gen.generate(inputs=input_ids, max_length=20, num_beams=7, no_repeat_ngram_size=2)[0],
            skip_special_tokens=False,
        )
    )
    print(
        tokenizer.decode(
            model.generate(inputs=input_ids, max_length=20, num_beams=7, no_repeat_ngram_size=2)[0],
            skip_special_tokens=False,
        )
    )

start = time()
for _ in range(3):
    model_gen.generate(inputs=input_ids, max_length=500, num_beams=5, no_repeat_ngram_size=2, min_length=500)
print(time() - start)

model.config.use_cache = True
with torch.inference_mode():
    start = time()
    for _ in range(3):
        model.generate(inputs=input_ids, max_length=500, num_beams=5, no_repeat_ngram_size=2, min_length=500)
    print(time() - start)

model = model.cpu()
del enc_onnx
del dec_onnx

trt_logger: Logger = trt.Logger(trt.Logger.ERROR)
runtime: Runtime = trt.Runtime(trt_logger)
trt_model_name = "trt-t5-dec.plan"

# create only of does not exist because it's slow to run...

# 768 for base model, 512 for small, make it dependent from the Pytorch model configuration
input_id_shape = TensorRTShape(min_shape=[5, 1], optimal_shape=[5, 500], max_shape=[5, 500], input_name="input_ids")
encoder_hidden_states_shape = TensorRTShape(
    min_shape=[5, 1, 512], optimal_shape=[5, 500 // 2, 512], max_shape=[5, 500, 512], input_name="encoder_hidden_states"
)


model = model.cuda()
model_onnx: ModelProto = onnx.load("test-dec.onnx")
model_onnx_all_nodes = add_output_nodes(model=model_onnx)
onnx_graph: Dict[str, Set[str]] = get_adjency_dict(model=model_onnx)
ort_model_all_nodes = create_model_for_provider(model_onnx_all_nodes.SerializeToString(), "CUDAExecutionProvider")


# use info from tokenizer size and max shape provided through the command line
def get_random_input():
    input = torch.randint(high=tokenizer.vocab_size, size=(5, 500), dtype=torch.int32, device="cuda")
    hidden_state = model.encoder(input_ids=input).last_hidden_state.detach().cpu().numpy()
    return {"input_ids": input.detach().cpu().numpy(), "encoder_hidden_states": hidden_state}


keep_fp32 = get_list_fp32_nodes(
    onnx_graph=onnx_graph, model=ort_model_all_nodes, get_input=get_random_input, nb_try=200
)
model = model.cpu()


<pad> Studien studies show that</s>
<pad> Studien studies show that</s>
11.085162878036499
14.284526824951172


In [5]:
engine: ICudaEngine = build_engine(
    runtime=runtime,
    onnx_file_path="test-dec.onnx",
    logger=trt_logger,
    workspace_size=20000 * 1024**2,
    fp16=True,
    int8=False,
    input_shapes=[input_id_shape, encoder_hidden_states_shape],
    fp16_fix=get_fix_fp16_network_func(keep_fp32=keep_fp32),
)
save_engine(engine, trt_model_name)

tensorrt_model = load_engine(runtime=runtime, engine_file_path=trt_model_name)
a = tensorrt_model(
    {
        "input_ids": input_ids.type(torch.int32).repeat((5, 1)),
        "encoder_hidden_states": out_enc.last_hidden_state.repeat((5, 1, 1)),
    }
)
print(a[0])

benchmark_input = torch.ones((5, 500), dtype=torch.int32, device="cuda")
benchmark_enc_output = out_enc.last_hidden_state.repeat((5, 1, 1))
for _ in range(10):
    tensorrt_model(
        {
            "input_ids": benchmark_input,
            "encoder_hidden_states": benchmark_enc_output,
        }
    )
start = time()
for _ in range(100):
    tensorrt_model(
        {
            "input_ids": benchmark_input,
            "encoder_hidden_states": benchmark_enc_output,
        }
    )
print(time() - start)

dec_onnx = create_model_for_provider("test-dec-opt.onnx", "CUDAExecutionProvider")
dec_onnx_out = decoder_onnx_inference(input_ids=input_ids, last_hidden_state=out_enc.last_hidden_state)


for _ in range(10):
    decoder_onnx_inference(input_ids=benchmark_input, last_hidden_state=benchmark_enc_output)
start = time()
for _ in range(100):
    decoder_onnx_inference(input_ids=benchmark_input, last_hidden_state=benchmark_enc_output)
print(time() - start)

model.cuda()
for _ in range(10):
    model.decoder(input_ids=benchmark_input, encoder_hidden_states=benchmark_enc_output)
start = time()
for _ in range(100):
    model.decoder(input_ids=benchmark_input, encoder_hidden_states=benchmark_enc_output)
print(time() - start)

# TensorRT, ONNX Runtime, Pytorch

# sequence 500
# 0.8640644550323486
# 0.6695075035095215
# 1.1308434009552002

# sequence 250
# 0.9177014827728271
# 0.6861860752105713
# 1.1923034191131592

tensor([[-48.0803, -14.2901, -20.4682,  ..., -67.6375, -67.8819, -67.8281],
        [-54.2086, -14.2957, -21.5224,  ..., -66.5668, -66.9079, -66.7077],
        [-35.6367,  -5.6427, -15.6152,  ..., -49.5692, -49.8774, -49.7235],
        [-31.3282,  -3.7932, -11.2617,  ..., -43.0694, -43.3203, -43.1963]],
       device='cuda:0')
1.4864044189453125
0.6534547805786133
1.1358990669250488


In [6]:
out_enc: BaseModelOutputWithPastAndCrossAttentions = model.encoder(input_ids=input_ids)
model.decoder(input_ids=input_ids, encoder_hidden_states=out_enc.last_hidden_state, past_key_values=None).last_hidden_state[:,-1,:]

tensor([[ 2.7558e-02,  1.5035e-01, -1.0642e-01,  1.3436e-01, -2.7034e-03,
          1.0121e-01, -3.7072e-02, -1.1961e-01,  4.2827e-01, -6.8828e-02,
          4.7860e-03,  1.0388e-01, -1.9117e-01, -9.1395e-02,  8.0236e-01,
         -2.9004e-02, -8.3236e-03, -4.1821e-02,  5.1209e-02, -8.1611e-02,
          5.2725e-02,  7.5980e-02, -8.5656e-02,  2.2240e+00,  4.7778e-02,
         -7.0944e-02, -3.6483e-02,  2.8398e-01, -3.3781e-02, -1.3103e-01,
          4.6392e-02, -4.0270e-02, -5.6167e-02, -1.5688e-02,  4.1200e-01,
         -3.9407e-02, -1.1113e-01, -7.7969e-02, -1.3417e-01,  1.9564e-01,
         -4.9984e-02, -5.9329e-02,  2.2479e-01,  7.3668e-02, -5.7873e-02,
          1.4900e-02, -6.9493e-01,  1.7280e-02,  1.0609e-01,  9.2358e-02,
         -1.3377e-01, -5.1072e-02, -8.8388e-02,  1.4478e-01, -1.1261e-01,
          4.1928e-03,  5.3930e-02, -7.4811e-02,  5.8460e-02,  2.1837e-02,
          8.2831e-02,  9.2348e-02, -2.5649e-02, -8.6028e-02, -1.2986e-02,
         -4.3584e-02, -4.8884e-02, -3.

In [7]:
out_dec_pytorch = model.decoder(input_ids=input_ids[:, :-1], encoder_hidden_states=out_enc.last_hidden_state)
model.decoder(input_ids=input_ids[:, -1:], encoder_hidden_states=out_enc.last_hidden_state, past_key_values=out_dec_pytorch.past_key_values).last_hidden_state[:,-1,:]

tensor([[ 2.7559e-02,  1.5035e-01, -1.0642e-01,  1.3436e-01, -2.7034e-03,
          1.0121e-01, -3.7072e-02, -1.1961e-01,  4.2827e-01, -6.8828e-02,
          4.7861e-03,  1.0388e-01, -1.9117e-01, -9.1395e-02,  8.0236e-01,
         -2.9004e-02, -8.3238e-03, -4.1821e-02,  5.1210e-02, -8.1611e-02,
          5.2724e-02,  7.5980e-02, -8.5656e-02,  2.2240e+00,  4.7778e-02,
         -7.0944e-02, -3.6484e-02,  2.8398e-01, -3.3781e-02, -1.3103e-01,
          4.6392e-02, -4.0270e-02, -5.6167e-02, -1.5688e-02,  4.1200e-01,
         -3.9406e-02, -1.1113e-01, -7.7969e-02, -1.3417e-01,  1.9564e-01,
         -4.9984e-02, -5.9328e-02,  2.2479e-01,  7.3668e-02, -5.7874e-02,
          1.4900e-02, -6.9493e-01,  1.7280e-02,  1.0609e-01,  9.2357e-02,
         -1.3377e-01, -5.1072e-02, -8.8387e-02,  1.4478e-01, -1.1260e-01,
          4.1932e-03,  5.3930e-02, -7.4811e-02,  5.8460e-02,  2.1838e-02,
          8.2830e-02,  9.2348e-02, -2.5650e-02, -8.6029e-02, -1.2986e-02,
         -4.3583e-02, -4.8884e-02, -3.

In [8]:
from itertools import chain
from transformers.onnx.features import FeaturesManager

feature = "seq2seq-lm-with-past"
model = FeaturesManager.get_model_from_feature(feature, model_name)
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(model, feature=feature)
onnx_config = model_onnx_config(model.config)

with torch.no_grad():
    model.config.return_dict = True
    model.eval()

    # Check if we need to override certain configuration item
    if onnx_config.values_override is not None:
        for override_config_key, override_config_value in onnx_config.values_override.items():
            setattr(model.config, override_config_key, override_config_value)

    # Ensure inputs match
    model_inputs = onnx_config.generate_dummy_inputs(tokenizer, framework=TensorType.PYTORCH)
    for k, v in model_inputs.items():
        if isinstance(v, torch.Tensor):
            model_inputs[k] = model_inputs[k].type(torch.int32)
    onnx_outputs = list(onnx_config.outputs.keys())

    onnx_config.patch_ops()

    # export can works with named args but the dict containing named args as to be last element of the args tuple
    torch.onnx.export(
        model,
        (model_inputs,),
        f="test-dec-cache.onnx",
        input_names=list(onnx_config.inputs.keys()),
        output_names=onnx_outputs,
        dynamic_axes={name: axes for name, axes in chain(onnx_config.inputs.items(), onnx_config.outputs.items())},
        do_constant_folding=True,
        use_external_data_format=onnx_config.use_external_data_format(model.num_parameters()),
        enable_onnx_checker=True,
        opset_version=13,
    )

    onnx_config.restore_ops()


/home/geantvert/.local/share/virtualenvs/fast_transformer/lib/python3.9/site-packages/torch/onnx/utils.py:90: UserWarning: 'enable_onnx_checker' is deprecated and ignored. It will be removed in the next PyTorch release. To proceed despite ONNX checker failures, catch torch.onnx.ONNXCheckerError.
  warnings.warn("'enable_onnx_checker' is deprecated and ignored. It will be removed in "
/home/geantvert/.local/share/virtualenvs/fast_transformer/lib/python3.9/site-packages/torch/onnx/utils.py:103: UserWarning: `use_external_data_format' is deprecated and ignored. Will be removed in next PyTorch release. The code will work as it is False if models are not larger than 2GB, Otherwise set to False because of size limits imposed by Protocol Buffers.
  warnings.warn("`use_external_data_format' is deprecated and ignored. Will be removed in next "


In [9]:
model.cuda()
out_dec_pytorch = model.decoder(input_ids=input_ids[:, :-1], encoder_hidden_states=out_enc.last_hidden_state)

model.cuda()

model_inputs = {
    'input_ids' : input_ids[:, -1:].type(torch.int32),
    'encoder_hidden_states': out_enc.last_hidden_state,
    'past_key_values': out_dec_pytorch.past_key_values,
}

input_names = ['input_ids',
 'encoder_hidden_states',
 'past_key_values.0.decoder.key',
 'past_key_values.0.decoder.value',
 'past_key_values.0.encoder.key',
 'past_key_values.0.encoder.value',
 'past_key_values.1.decoder.key',
 'past_key_values.1.decoder.value',
 'past_key_values.1.encoder.key',
 'past_key_values.1.encoder.value',
 'past_key_values.2.decoder.key',
 'past_key_values.2.decoder.value',
 'past_key_values.2.encoder.key',
 'past_key_values.2.encoder.value',
 'past_key_values.3.decoder.key',
 'past_key_values.3.decoder.value',
 'past_key_values.3.encoder.key',
 'past_key_values.3.encoder.value',
 'past_key_values.4.decoder.key',
 'past_key_values.4.decoder.value',
 'past_key_values.4.encoder.key',
 'past_key_values.4.encoder.value',
 'past_key_values.5.decoder.key',
 'past_key_values.5.decoder.value',
 'past_key_values.5.encoder.key',
 'past_key_values.5.encoder.value']

output_names = ['logits',
 'present.0.decoder.key',
 'present.0.decoder.value',
 'present.0.encoder.key',
 'present.0.encoder.value',
 'present.1.decoder.key',
 'present.1.decoder.value',
 'present.1.encoder.key',
 'present.1.encoder.value',
 'present.2.decoder.key',
 'present.2.decoder.value',
 'present.2.encoder.key',
 'present.2.encoder.value',
 'present.3.decoder.key',
 'present.3.decoder.value',
 'present.3.encoder.key',
 'present.3.encoder.value',
 'present.4.decoder.key',
 'present.4.decoder.value',
 'present.4.encoder.key',
 'present.4.encoder.value',
 'present.5.decoder.key',
 'present.5.decoder.value',
 'present.5.encoder.key',
 'present.5.encoder.value']

dynamic_axis = {'input_ids': {0: 'batch', 1: 'encoder_sequence'}, 'past_key_values.0.decoder.key': {0: 'batch', 2: 'past_decoder_sequence'}, 'past_key_values.0.decoder.value': {0: 'batch', 2: 'past_decoder_sequence'}, 'past_key_values.0.encoder.key': {0: 'batch', 2: 'past_encoder_sequence'}, 'past_key_values.0.encoder.value': {0: 'batch', 2: 'past_encoder_sequence'}, 'past_key_values.1.decoder.key': {0: 'batch', 2: 'past_decoder_sequence'}, 'past_key_values.1.decoder.value': {0: 'batch', 2: 'past_decoder_sequence'}, 'past_key_values.1.encoder.key': {0: 'batch', 2: 'past_encoder_sequence'}, 'past_key_values.1.encoder.value': {0: 'batch', 2: 'past_encoder_sequence'}, 'past_key_values.2.decoder.key': {0: 'batch', 2: 'past_decoder_sequence'}, 'past_key_values.2.decoder.value': {0: 'batch', 2: 'past_decoder_sequence'}, 'past_key_values.2.encoder.key': {0: 'batch', 2: 'past_encoder_sequence'}, 'past_key_values.2.encoder.value': {0: 'batch', 2: 'past_encoder_sequence'}, 'past_key_values.3.decoder.key': {0: 'batch', 2: 'past_decoder_sequence'}, 'past_key_values.3.decoder.value': {0: 'batch', 2: 'past_decoder_sequence'}, 'past_key_values.3.encoder.key': {0: 'batch', 2: 'past_encoder_sequence'}, 'past_key_values.3.encoder.value': {0: 'batch', 2: 'past_encoder_sequence'}, 'past_key_values.4.decoder.key': {0: 'batch', 2: 'past_decoder_sequence'}, 'past_key_values.4.decoder.value': {0: 'batch', 2: 'past_decoder_sequence'}, 'past_key_values.4.encoder.key': {0: 'batch', 2: 'past_encoder_sequence'}, 'past_key_values.4.encoder.value': {0: 'batch', 2: 'past_encoder_sequence'}, 'past_key_values.5.decoder.key': {0: 'batch', 2: 'past_decoder_sequence'}, 'past_key_values.5.decoder.value': {0: 'batch', 2: 'past_decoder_sequence'}, 'past_key_values.5.encoder.key': {0: 'batch', 2: 'past_encoder_sequence'}, 'past_key_values.5.encoder.value': {0: 'batch', 2: 'past_encoder_sequence'}, 'logits': {0: 'batch', 1: 'decoder_sequence'}, 'present.0.decoder.key': {0: 'batch', 2: 'past_decoder_sequence + sequence'}, 'present.0.decoder.value': {0: 'batch', 2: 'past_decoder_sequence + sequence'}, 'present.0.encoder.key': {0: 'batch', 2: 'past_encoder_sequence'}, 'present.0.encoder.value': {0: 'batch', 2: 'past_encoder_sequence'}, 'present.1.decoder.key': {0: 'batch', 2: 'past_decoder_sequence + sequence'}, 'present.1.decoder.value': {0: 'batch', 2: 'past_decoder_sequence + sequence'}, 'present.1.encoder.key': {0: 'batch', 2: 'past_encoder_sequence'}, 'present.1.encoder.value': {0: 'batch', 2: 'past_encoder_sequence'}, 'present.2.decoder.key': {0: 'batch', 2: 'past_decoder_sequence + sequence'}, 'present.2.decoder.value': {0: 'batch', 2: 'past_decoder_sequence + sequence'}, 'present.2.encoder.key': {0: 'batch', 2: 'past_encoder_sequence'}, 'present.2.encoder.value': {0: 'batch', 2: 'past_encoder_sequence'}, 'present.3.decoder.key': {0: 'batch', 2: 'past_decoder_sequence + sequence'}, 'present.3.decoder.value': {0: 'batch', 2: 'past_decoder_sequence + sequence'}, 'present.3.encoder.key': {0: 'batch', 2: 'past_encoder_sequence'}, 'present.3.encoder.value': {0: 'batch', 2: 'past_encoder_sequence'}, 'present.4.decoder.key': {0: 'batch', 2: 'past_decoder_sequence + sequence'}, 'present.4.decoder.value': {0: 'batch', 2: 'past_decoder_sequence + sequence'}, 'present.4.encoder.key': {0: 'batch', 2: 'past_encoder_sequence'}, 'present.4.encoder.value': {0: 'batch', 2: 'past_encoder_sequence'}, 'present.5.decoder.key': {0: 'batch', 2: 'past_decoder_sequence + sequence'}, 'present.5.decoder.value': {0: 'batch', 2: 'past_decoder_sequence + sequence'}, 'present.5.encoder.key': {0: 'batch', 2: 'past_encoder_sequence'}, 'present.5.encoder.value': {0: 'batch', 2: 'past_encoder_sequence'}}

with torch.no_grad():
    model.config.return_dict = True
    model.eval()

    # export can works with named args but the dict containing named args as to be last element of the args tuple
    torch.onnx.export(
        model.decoder,
        (model_inputs,),
        f="test-dec-cache.onnx",
        input_names=input_names,
        output_names=output_names,
        dynamic_axes=dynamic_axis,
        do_constant_folding=True,
        use_external_data_format=False,
        enable_onnx_checker=True,
        opset_version=13,
    )

    onnx_config.restore_ops()

In [10]:
ort_cache = create_model_for_provider("test-dec-cache.onnx", "CPUExecutionProvider")

In [11]:
input_cache = dict()
input_cache["input_ids"] = input_ids[:, -1:].type(torch.int32).detach().cpu().numpy()

for index, (k_dec, v_dec, k_enc, v_enc) in enumerate(out_dec_pytorch.past_key_values):  # type: int, (torch.Tensor, torch.Tensor)
    input_cache[f"past_key_values.{index}.decoder.key"] = k_dec.detach().cpu().numpy()
    input_cache[f"past_key_values.{index}.decoder.value"] = v_dec.detach().cpu().numpy()
    input_cache[f"past_key_values.{index}.encoder.key"] = k_enc.detach().cpu().numpy()
    input_cache[f"past_key_values.{index}.encoder.value"] = v_enc.detach().cpu().numpy()

ort_cache.run(["logits"], input_cache)[0].shape

(1, 1, 512)

In [12]:
ort_cache = create_model_for_provider("test-dec-cache.onnx", "CUDAExecutionProvider")
input_cache = dict()
input_cache["input_ids"] = input_ids[:, -1:]

for index, (k_dec, v_dec, k_enc, v_enc) in enumerate(out_dec_pytorch.past_key_values):  # type: int, (torch.Tensor, torch.Tensor, torch.Tensor)
    input_cache[f"past_key_values.{index}.decoder.key"] = k_dec.cuda()
    input_cache[f"past_key_values.{index}.decoder.value"] = v_dec.cuda()
    input_cache[f"past_key_values.{index}.encoder.key"] = k_enc.cuda()
    input_cache[f"past_key_values.{index}.encoder.value"] = v_enc.cuda()


inference_onnx_binding(model_onnx=ort_cache, inputs=input_cache, device="cuda", output_shape={"logits": (1, 1, 512)})["logits"]

tensor([[[ 2.7551e-02,  1.5033e-01, -1.0638e-01,  1.3437e-01, -2.7034e-03,
           1.0119e-01, -3.7066e-02, -1.1958e-01,  4.2827e-01, -6.8837e-02,
           4.7930e-03,  1.0388e-01, -1.9117e-01, -9.1408e-02,  8.0224e-01,
          -2.9010e-02, -8.3016e-03, -4.1842e-02,  5.1235e-02, -8.1596e-02,
           5.2729e-02,  7.5967e-02, -8.5656e-02,  2.2240e+00,  4.7762e-02,
          -7.0900e-02, -3.6481e-02,  2.8401e-01, -3.3791e-02, -1.3101e-01,
           4.6434e-02, -4.0252e-02, -5.6140e-02, -1.5710e-02,  4.1203e-01,
          -3.9375e-02, -1.1114e-01, -7.7971e-02, -1.3421e-01,  1.9566e-01,
          -4.9996e-02, -5.9346e-02,  2.2478e-01,  7.3680e-02, -5.7880e-02,
           1.4898e-02, -6.9503e-01,  1.7287e-02,  1.0606e-01,  9.2383e-02,
          -1.3378e-01, -5.1050e-02, -8.8404e-02,  1.4487e-01, -1.1263e-01,
           4.1829e-03,  5.3950e-02, -7.4830e-02,  5.8463e-02,  2.1883e-02,
           8.2851e-02,  9.2368e-02, -2.5662e-02, -8.6041e-02, -1.2973e-02,
          -4.3556e-02, -4